# Cloud Infrastructure

# ======================== Homework 2.1 ========================

## 1). Write a program to implement the Maekawa’s algorithm. The inputs are N- the number of nodes and K- the size of each subset and the output is a list of subsets that have pare-wised non-null intersection. 

In [15]:
from itertools import combinations

In [217]:
def generate_combinations(N,K):
    combined_set = combinations([i for i in range(1,N+1)],K)
    combined_set = [set(comb) for comb in list(combined_set)]
    return combined_set

def is_intersection_present(temp_arr,cbn):
    is_intersection = True
    for temp in temp_arr:
        if len(temp.intersection(cbn)) > 1:
            is_intersection = False
            break;
        if not temp.intersection(cbn):
            is_intersection = False
            break;
    return is_intersection

def create_quorum(N,K):
    combination_sets = generate_combinations(N,K)

    itr = 1
    temp_arr = [combination_sets.pop(0)]
    for c in combination_sets:
        if itr <= N:
            if is_intersection_present(temp_arr, c):
                temp_arr.append(c)
                itr = itr + 1
    missing_nums_arr = []
    pr_quorum = {}
    for i in range(1,N+1):
        status = False
        for q in temp_arr:
            if (i in q):
                pr_quorum[i] = q
                temp_arr.remove(q)
                status = True
                break
        if not status:
            missing_nums_arr.append(i)
    if missing_nums_arr:
        for num_m in missing_nums_arr:
            for qm in temp_arr:
                for pr in qm:
                    if (num_m in pr_quorum.get(pr)):
                        pr_quorum[num_m] = pr_quorum[pr]
                        pr_quorum[pr] = qm
                        temp_arr.remove(qm)
                        missing_nums_arr.remove(num_m)
                        break
    return pr_quorum

In [218]:
pr_quorum = create_quorum(7,3)
print("Process : Quorum ",pr_quorum)

Process : Quorum  {1: {1, 2, 3}, 2: {2, 4, 6}, 3: {3, 5, 6}, 4: {1, 4, 5}, 5: {2, 5, 7}, 6: {1, 6, 7}, 7: {3, 4, 7}}


## 2).Create N processes and use quorum-based algorithm with Maekawa voting sets to achieve distributed mutual exclusion.

In [223]:
import threading
import time
from collections import deque

class critical_section:
    p_quorum_dict = {}
    pr_queue = {}
    
    def __init__(self, pr_quorum):
        self.p_quorum_dict = pr_quorum
        self._lock = threading.Lock()
        for k in pr_quorum.keys():
            self.pr_queue[k] = deque([])
    
    # On receipt of request for pi to pj
    def request_quorum(self,pid):
        print("=>access to quorum {} requested by process {}".format(self.p_quorum_dict.get(pid),pid))
        quorum = self.p_quorum_dict.get(pid)
        for proc in quorum:
            self.pr_queue.get(proc).append([pid,"REQUEST"])
            
    # To enter Critical section: enter the critical section if it has received the 
    # REPLY message from all the site in request set 
    def enter_cs(self, pid):
        retry,count = 5,0
        self.request_quorum(pid)
       
        if self.quorum_reply(pid):
            with self._lock:
                print("=>entering critical section : process {}".format(pid))
                time.sleep(2)
    
    def quorum_reply(self,pid):
        quorum = self.p_quorum_dict.get(pid)
        quorum_len = len(quorum)
        itr = 0
        for process in quorum:
            process_q =self.pr_queue.get(process)
            if process_q:
                element = process_q[0]
                if element[0] == pid:
                    element[1] ="VOTED"
                    itr = itr + 1
        if itr == quorum_len:
            for process in quorum:
                self.pr_queue.get(process).popleft()
            print("=>quorum granted access to process {}".format(pid))
            return True
        
        return False

In [224]:
cs = critical_section(pr_quorum)
threads = []

# starting all the thread at once
for p in pr_quorum.keys():
    thd = threading.Thread(target=cs.enter_cs, args=(p,))
    threads.append(thd)
    thd.start()

for tr in threads:
    tr.join()

=>access to quorum {1, 2, 3} requested by process 1
=>quorum granted access to process 1=>access to quorum {2, 4, 6} requested by process 2
=>quorum granted access to process 2
=>entering critical section : process 2

=>access to quorum {3, 5, 6} requested by process 3
=>quorum granted access to process 3
=>access to quorum {1, 4, 5} requested by process 4
=>quorum granted access to process 4
=>access to quorum {2, 5, 7} requested by process 5
=>quorum granted access to process 5
=>access to quorum {1, 6, 7} requested by process 6
=>quorum granted access to process 6
=>access to quorum {3, 4, 7} requested by process 7
=>quorum granted access to process 7
=>entering critical section : process 1
=>entering critical section : process 3
=>entering critical section : process 4
=>entering critical section : process 5
=>entering critical section : process 6
=>entering critical section : process 7


# 3). Discuss the possibility of deadlock in the implementation. 

Maekawa’s algorithm is deadlock prone because a site is exclusively locked by other sites and requests are not prioritized by their timestamps.
- Assume three sites Si, Sj, and Sk simultaneously invoke mutual exclusion.
- Suppose Ri ∩ Rj= {Sij}, Rj ∩ Rk = {Sjk}, and Rk ∩ Ri= {Ski}.

Consider the following scenario:
-  Sij has been locked by Si (forcing Sj to wait at Sij ).
-  Sjk has been locked by Sj (forcing Sk to wait at Sjk ).
-  Ski has been locked by Sk (forcing Si to wait at Ski ).

This state represents a deadlock involving sites Si, Sj, and Sk .


# ======================== Homework 2.2 ========================

## Min - Min Algorithm

In [6]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('bag_of_tasks.xlsx', sheet_name='Sheet1')

In [7]:
df

,T0,T1,T2,T3,T4,T5,T6,T7
M0,13,31,17.0,19,94.0,8,10,80
M1,79,13,1.0,28,75.0,24,57,17
M2,23,14,NaN,10,NaN,3,13,38
M3,71,94,23.0,4,58.0,32,1,40
M4,60,60,36.0,58,NaN,4,92,66
M5,27,61,8.0,73,68.0,94,75,25
M6,2,57,86.0,40,46.0,89,29,88


In [8]:
import math
min_exec_time_dic = {'M0' : [],'M1' : [], 'M2' : [], 'M3': [],'M4' : [], 'M5' : [], 'M6' : []}
min_exe=[]
task_1 = ''
index_1  = ''

debug = False

while not df.empty:
    for task in df.columns:
        for index in df.index:
            try:
                if (not math.isnan(df[task_1][index_1])) and min_exe[0] > df[task][index]:
                    del min_exe[:]
                    min_exe.append(df[task][index])
                    task_1 = task
                    index_1 = index
            except:
                min_exe.append(df[task][index])
                task_1 = task
                index_1 = index
    min_exec_time_dic[index_1].append(task_1)
    print(f"{task_1} is assigned to {index_1} with Time {df[task_1][index_1]}")
    if (not math.isnan(df[task_1][index_1])):
        df.loc[index_1] = df.loc[index_1] + df[task_1][index_1]
        del df[task_1]
        del min_exe[:]

print("Final Schedule", min_exec_time_dic)

T2 is assigned to M1 with Time 1.0
T6 is assigned to M3 with Time 1.0
T0 is assigned to M6 with Time 2.0
T5 is assigned to M2 with Time 3.0
T3 is assigned to M3 with Time 5.0
T1 is assigned to M1 with Time 14.0
T7 is assigned to M5 with Time 25.0
T4 is assigned to M6 with Time 48.0
Final Schedule {'M0': [], 'M1': ['T2', 'T1'], 'M2': ['T5'], 'M3': ['T6', 'T3'], 'M4': [], 'M5': ['T7'], 'M6': ['T0', 'T4']}


## Max - Min Algorithm

In [12]:
maxmindf = pd.read_excel('bag_of_tasks.xlsx', sheet_name='Sheet1')

In [13]:
maxmindf

,T0,T1,T2,T3,T4,T5,T6,T7
M0,13,31,17.0,19,94.0,8,10,80
M1,79,13,1.0,28,75.0,24,57,17
M2,23,14,NaN,10,NaN,3,13,38
M3,71,94,23.0,4,58.0,32,1,40
M4,60,60,36.0,58,NaN,4,92,66
M5,27,61,8.0,73,68.0,94,75,25
M6,2,57,86.0,40,46.0,89,29,88


In [14]:
# Program to schedule above bag of task using Max-Min
min_exe_dict = {}

final_assignment_dict = {'M0' : [],'M1' : [], 'M2' : [], 'M3': [],'M4' : [], 'M5' : [], 'M6' : []}

min_exe_1 = []

task_1 = ''
index_1  = ''

while not maxmindf.empty:
    for task in maxmindf.columns:
        for index in maxmindf.index:
            try:
                if (not math.isnan(maxmindf[task][index])) and min_exe_1[0] > maxmindf[task][index]:
                    del min_exe_1[:]
                    min_exe_1.append(maxmindf[task][index])
                    task_1 = task
                    index_1 = index
            except:
                 min_exe_1.append(maxmindf[task][index])
                 task_1 = task
                 index_1 = index
        min_exe_dict[task_1+'-'+index_1] = maxmindf[task_1][index_1]
        del min_exe_1[:]
    t, m = max(min_exe_dict, key=min_exe_dict.get).split('-')
    maxmindf.loc[m] = maxmindf.loc[m] + maxmindf[t][m]
    final_assignment_dict[m].append(t)
    print(f"{t} is assigned to {m} with Time {maxmindf[t][m]}")
    del maxmindf[t]
    min_exe_dict.clear()

print("Final_Schedule",final_assignment_dict)

T4 is assigned to M6 with Time 92.0
T7 is assigned to M1 with Time 34.0
T1 is assigned to M2 with Time 28.0
T0 is assigned to M0 with Time 26.0
T2 is assigned to M5 with Time 16.0
T3 is assigned to M3 with Time 8.0
T6 is assigned to M3 with Time 10.0
T5 is assigned to M4 with Time 8.0
Final_Schedule {'M0': ['T0'], 'M1': ['T7'], 'M2': ['T1'], 'M3': ['T3', 'T6'], 'M4': ['T5'], 'M5': ['T2'], 'M6': ['T4']}


# Sufferage Algorithm

In [15]:
sufferage_df = pd.read_excel('bag_of_tasks.xlsx', sheet_name='Sheet1')

In [16]:
sufferage_df

,T0,T1,T2,T3,T4,T5,T6,T7
M0,13,31,17.0,19,94.0,8,10,80
M1,79,13,1.0,28,75.0,24,57,17
M2,23,14,NaN,10,NaN,3,13,38
M3,71,94,23.0,4,58.0,32,1,40
M4,60,60,36.0,58,NaN,4,92,66
M5,27,61,8.0,73,68.0,94,75,25
M6,2,57,86.0,40,46.0,89,29,88


In [17]:
# Program to schedule above bag of task using Sufferage
min_exe_suf = []
min_exe_suf_dict = {}
min_exe_suf_dict_temp = {}

diff = 0
task_1 = ''
index_1  = ''

final_assignment_suff = {'M0' : [],'M1' : [], 'M2' : [], 'M3': [],'M4' : [], 'M5' : [], 'M6' : []}
while not sufferage_df.empty:
    for task in sufferage_df.columns:
        for index in sufferage_df.index:
            try:
                if (not math.isnan(sufferage_df[task][index])):
                    diff = abs(min_exe_suf[0] - sufferage_df[task][index])
                    del min_exe_suf[:]
                    min_exe_suf.append(diff)
                    task_1 = task
                    index_1 = index
            except:
                min_exe_suf.append(sufferage_df[task][index])
                task_1 = task
                index_1 = index

        min_exe_suf_dict_temp[task_1] = diff
        del min_exe_suf[:]
    max_diff_task = max(min_exe_suf_dict_temp , key=min_exe_suf_dict_temp.get)
    machine_max = sufferage_df[max_diff_task].idxmin()
    final_assignment_suff[machine_max].append(max_diff_task)
    print(f"{max_diff_task} is assigned to {machine_max} with Time {sufferage_df[max_diff_task][machine_max]}")
    sufferage_df.loc[machine_max] = sufferage_df.loc[machine_max] + sufferage_df[max_diff_task][machine_max]
    del min_exe_suf[:]
    del sufferage_df[max_diff_task]
    min_exe_suf_dict_temp.clear()

print("Final Schedule : ",final_assignment_suff)

T2 is assigned to M1 with Time 1.0
T7 is assigned to M1 with Time 18.0
T4 is assigned to M6 with Time 46.0
T1 is assigned to M2 with Time 14.0
T3 is assigned to M3 with Time 4.0
T6 is assigned to M3 with Time 5.0
T5 is assigned to M4 with Time 4.0
T0 is assigned to M0 with Time 13.0
Final Schedule :  {'M0': ['T0'], 'M1': ['T2', 'T7'], 'M2': ['T1'], 'M3': ['T3', 'T6'], 'M4': ['T5'], 'M5': [], 'M6': ['T4']}
